# Create video of the images in the folder

### Import libraries

In [1]:
import cv2
import os
import numpy as np

### Set paths and read images

In [2]:
# Set paths and video output name
image_folder = 'data/Varrio_Scanner2'
video_name = 'Video_Varrio2_V2.mp4'

# Set font size
font_size=3

# Number of pictures in second
fps=3

# Cut out number of edge pixels
cut = 50

### Get image paths and set video settings

In [3]:
# Get image names
images = [img for img in os.listdir(image_folder) if img.endswith(".jpg")]

# Sort image names
images = sorted(images)

# Set video frame size to 2338x1653 (Scale = 0.5). Orig size 4676 × 3306.
# We must use fixed size, because we can not know size of 
# the pics taken in the future and size is used in parameter tuning.
width = int(2338)
height = int(1653)
dim_vid=(width-cut, (height-cut)*2)
dim_pic=(width, height)

# Set video settings
video = cv2.VideoWriter(filename=video_name, fourcc=cv2.VideoWriter_fourcc(*'XVID'), fps=fps, frameSize=dim_vid)

# Set text font
font = cv2.FONT_HERSHEY_SIMPLEX

### Create only root images

In [4]:
# Detect roots from image
def get_roots(img_color):
    
    # Convert to gray scale
    gray = cv2.cvtColor(img_color, cv2.COLOR_BGR2GRAY)

    # Remove noise with gaussian blur (3, 3 works)
    edges = cv2.GaussianBlur(gray,(3,3),0)

    # Edges to binary (black and white) (180 and 250)
    ret,edges = cv2.threshold(edges,180,256,cv2.THRESH_BINARY)

    # Remove isolated pixels
    edges = cv2.GaussianBlur(edges,(5,5),0)
    edges = cv2.threshold(edges, 150 , 250, cv2.THRESH_BINARY)
    new_roots = edges[1]
    
    # Remove isolated pixels
    #new_roots = remove_tiny_areas(new_roots)

    # Return results
    return new_roots

# Filter noise by the pixel area
def remove_tiny_areas(im):
    # Run canny edge detection
    edges = cv2.Canny(im, 50, 250)
    
    # Get contrours
    cnts, hierarchy = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    # Loop over our contours. Remove controus with area less than 10.
    contours=[]
    for c in cnts:
        size = cv2.contourArea(c)
        if size > 10:
            contours.append(c)
            
    # make empty pic where to save contours
    height, width = im.shape
    empty=np.zeros((height,width), np.uint8)
            
    # Turn contours to image format
    contours = cv2.drawContours(empty, contours, -1, (0,255,0), 3)
    
    return contours

### Convert pics to video and save output

In [5]:
# Initialize roots locations binary array
roots=np.zeros((height-2*cut,width-2*cut), np.uint8)

# Make video of images
for image in images:
    
    # Read image
    im = cv2.imread(os.path.join(image_folder, image))
    
    # Resize image
    im = cv2.resize(im, dim_pic,interpolation = cv2.INTER_AREA)
    
    # Cut out border pixels from each side, because they have lots of noise.
    im = im[cut:, cut:]
    im = im[:-cut, :-cut]
    
    # Get new root location pixels
    new_roots=get_roots(im)
    
    # Add new root locations pixels to previous root locations pixels
    roots = new_roots+roots
    
    # Mask root locations to image (get colors to root pixels)
    im_masked = cv2.bitwise_or(im, im, mask=roots)
    
    # Concat original image and roots image vertically
    im = np.concatenate((im, im_masked), axis=0)
    
    # Get image date from the image name
    # Date is in different location for Hyde and Varrio
    #date = image[20:30] # For Hyde Scanners
    date = image[14:24] # For Varrio Scanners
    
    # Add date to upper left corner
    cv2.putText(img=im, text=date, org=(font_size*10,font_size*30), fontFace=font, fontScale=font_size, 
                color=(255, 255, 255), thickness=font_size, lineType=cv2.LINE_AA)
    
    # Add frame to video
    video.write(im)
     
# Close windows
cv2.destroyAllWindows() 

# Save output video
video.release()